In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import SAM
import numpy as np

model = SAM('sam_b.pt')

image_path = './data/test_img_R.jpg' 
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
results = model(image_rgb)



In [ ]:
if results and results[0].masks is not None:
    # Get the raw masks as a numpy array (HxWxN, where N is number of masks)
    masks_tensor = results[0].masks.data
    masks_np = masks_tensor.cpu().numpy() # Move to CPU and convert to NumPy

    # Create a blank image to draw all masks on
    masked_image = np.zeros_like(image_rgb, dtype=np.uint8)

    # You can iterate through masks and draw them with random colors
    for i, mask in enumerate(masks_np):
        color = np.random.randint(0, 255, 3, dtype=np.uint8)
        # Apply the mask: wherever mask is True, apply the color
        masked_image[mask] = color

    # You can blend the original image with the masks for a better visualization
    alpha = 0.5 # Transparency of the masks
    blended_image = cv2.addWeighted(image_rgb, 1, masked_image, alpha, 0)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(image_rgb)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(blended_image)
    plt.title("Segmented Image")
    plt.axis('off')
    plt.show()

    print(f"Found {len(masks_np)} segments.")

else:
    print("No segments found or an issue occurred during segmentation.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import os # For creating a directory to save images

# Create a directory to save the images
output_dir = "dbscan_demo_images"
os.makedirs(output_dir, exist_ok=True)

def euclidean_distance(point1, point2):
    """Calculates the Euclidean distance between two points."""
    return np.sqrt(np.sum((point1 - point2)**2))

def find_neighbors(data, point_idx, eps):
    """Finds all points within the epsilon radius of a given point."""
    neighbors = []
    for i in range(data.shape[0]):
        if i == point_idx: # A point is always its own neighbor for distance 0
            continue
        if euclidean_distance(data[point_idx], data[i]) <= eps:
            neighbors.append(i)
    return neighbors

def plot_current_state(data, labels, core_point_indices, title, filename, show_neighbor_counts=None):
    """Helper function to plot the current state of DBSCAN."""
    plt.figure(figsize=(10, 8))

    unique_labels = set(labels)
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))

    for k, col in zip(unique_labels, colors):
        if k == -1:  # Noise points
            col = 'k' # Black for noise
            marker = 'x'
            ms = 8
            label_text = 'Noise'
        else:
            marker = 'o'
            ms = 10 # Slightly larger for demo clarity
            label_text = f'Cluster {k}'

        class_member_mask = (labels == k)
        xy = data[class_member_mask]
        plt.plot(xy[:, 0], xy[:, 1], marker, markerfacecolor=col,
                 markeredgecolor='k', markersize=ms, label=label_text)
    
    # Highlight core points with a red circle outline
    if core_point_indices:
        core_points_data = data[list(core_point_indices)]
        plt.plot(core_points_data[:, 0], core_points_data[:, 1], 'o', markerfacecolor='none',
                 markeredgecolor='red', markersize=16, linewidth=2, label='Core Points', linestyle='none')
    
    # Add neighbor counts if provided
    if show_neighbor_counts is not None:
        for i, count in enumerate(show_neighbor_counts):
            if count is not None: # Only show for points where count was calculated
                plt.text(data[i, 0] + 0.1, data[i, 1] + 0.1, str(count), fontsize=9, color='blue', ha='left', va='bottom')


    plt.title(title)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend(loc='upper left', bbox_to_anchor=(1,1))
    plt.grid(True)
    plt.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.axvline(0, color='gray', linestyle='--', linewidth=0.5)
    plt.tight_layout() # Adjust layout to prevent labels from overlapping
    plt.savefig(os.path.join(output_dir, filename))
    plt.close() # Close the plot to free memory

def dbscan_step_demo(data, eps, min_pts):
    """
    DBSCAN demonstration saving images at key steps.
    """
    n_points = data.shape[0]
    labels = np.full(n_points, 0, dtype=int)  # 0 for unclassified initially
    visited = np.full(n_points, False, dtype=bool)
    cluster_id = 0
    core_point_indices = set()
    all_neighbor_counts = [None] * n_points # To store neighbor counts for demo display

    print(f"--- DBSCAN Process with eps={eps}, MinPts={min_pts} ---")

    # --- Step 0: Initial State ---
    plot_current_state(data, labels, core_point_indices, 
                       f'DBSCAN Demo: Initial State (eps={eps}, MinPts={min_pts})', 
                       'step_0_initial_state.png')
    print("Saved: step_0_initial_state.png")

    # --- Step 1: Identify All Core Points ---
    temp_core_point_indices = set()
    print("\n--- Phase 1: Identifying all potential Core Points ---")
    for i in range(n_points):
        print(f"Checking Point {i}: {data[i]}")
        current_neighbors = find_neighbors(data, i, eps)
        # Include the point itself in the count for core point definition
        current_neighbor_count = len(current_neighbors) + 1
        all_neighbor_counts[i] = current_neighbor_count # Store count for plotting

        if current_neighbor_count >= min_pts:
            print(f"  - Point {i} IS a **CORE POINT** (Neighbors: {current_neighbor_count} >= MinPts: {min_pts})")
            temp_core_point_indices.add(i)
        else:
            print(f"  - Point {i} is NOT a core point (Neighbors: {current_neighbor_count} < MinPts: {min_pts})")
    
    # Update actual core_point_indices after checking all points
    core_point_indices = temp_core_point_indices.copy()

    plot_current_state(data, labels, core_point_indices,
                       f'DBSCAN Demo: Identified All Core Points (eps={eps}, MinPts={min_pts})',
                       'step_1_all_core_points_identified.png',
                       show_neighbor_counts=all_neighbor_counts)
    print("Saved: step_1_all_core_points_identified.png")


    # --- Step 2: Cluster Expansion ---
    print("\n--- Phase 2: Expanding Clusters ---")
    first_cluster_expanded = False

    for i in range(n_points):
        if visited[i]:
            continue # Skip if already visited/assigned to a cluster

        visited[i] = True
        
        if i in core_point_indices: # Only core points can initiate a new cluster expansion
            cluster_id += 1
            labels[i] = cluster_id
            
            # Find neighbors again as find_neighbors excludes self
            current_neighbors_for_queue = find_neighbors(data, i, eps)
            queue = deque(current_neighbors_for_queue)

            print(f"\nExpanding Cluster {cluster_id} from Core Point {i}")

            while queue:
                current_expand_idx = queue.popleft()
                
                # If this point was unclassified noise (-1), mark it as visited and assign
                if labels[current_expand_idx] == 0: # Check if it's currently unclassified
                    visited[current_expand_idx] = True
                    labels[current_expand_idx] = cluster_id
                    print(f"  - Point {current_expand_idx} assigned to Cluster {cluster_id}.")

                    # If this newly assigned point is also a core point, add its neighbors to queue
                    if current_expand_idx in core_point_indices:
                        print(f"    - Point {current_expand_idx} is also a CORE POINT. Adding its unvisited neighbors to queue.")
                        expand_neighbors = find_neighbors(data, current_expand_idx, eps)
                        for neighbor_of_expand_idx in expand_neighbors:
                            if not visited[neighbor_of_expand_idx] and labels[neighbor_of_expand_idx] == 0: # Add only unvisited and unclassified points
                                queue.append(neighbor_of_expand_idx)
                
            # --- Save state after first cluster is formed ---
            if not first_cluster_expanded:
                plot_current_state(data, labels, core_point_indices,
                                   f'DBSCAN Demo: After First Cluster Formed (Cluster {cluster_id})',
                                   f'step_2_cluster_{cluster_id}_formed.png',
                                   show_neighbor_counts=all_neighbor_counts)
                print(f"Saved: step_2_cluster_{cluster_id}_formed.png")
                first_cluster_expanded = True # Flag to ensure this step is saved only once

        else: # Not a core point
            # If a point is not a core point and hasn't been visited by a core point's expansion, it's noise
            if labels[i] == 0: # If still unclassified (not assigned by another core point)
                labels[i] = -1 # Mark as noise
                print(f"  - Point {i} is not a core point and was not assigned. Marked as NOISE.")

    # --- Step 3: Final State ---
    # Any remaining 0 labels should become -1 (noise)
    labels[labels == 0] = -1

    plot_current_state(data, labels, core_point_indices,
                       f'DBSCAN Demo: Final Clustering Result (eps={eps}, MinPts={min_pts})',
                       'step_3_final_result.png',
                       show_neighbor_counts=all_neighbor_counts)
    print("Saved: step_3_final_result.png")

    print("\n--- DBSCAN Process Complete ---")
    return labels, core_point_indices

# --- Example Usage ---
if __name__ == "__main__":
    # 1. Generate a smaller, clearer sample data
    np.random.seed(0) # for reproducibility

    # Cluster 1
    c1 = np.array([[1, 1], [1.2, 1.2], [0.9, 0.8], [1.1, 0.9], [0.8, 1.1]])
    # Cluster 2
    c2 = np.array([[-1, -1], [-0.8, -1.2], [-1.1, -0.9], [-0.9, -0.8], [-1.2, -1.1]])
    # Border point for C1
    border_point_c1 = np.array([[1.5, 1.5]])
    # Noise point
    noise_point = np.array([[3, -3]])
    # Another small group that might be noise or another cluster depending on params
    c3 = np.array([[0, -2], [0.2, -2.1], [-0.1, -1.9]])


    data = np.vstack((c1, c2, border_point_c1, noise_point, c3))
    
    # 2. Set DBSCAN parameters
    # Adjust these to see how different core points and clusters form
    eps = 0.5  # Epsilon: radius of neighborhood
    min_pts = 4 # MinPts: minimum number of points in a neighborhood (including itself) to be a core point

    print(f"--- Running DBSCAN Demo with {len(data)} data points ---")
    print(f"Data Points:\n{data}")

    # 3. Run the DBSCAN demo
    labels, core_point_indices = dbscan_step_demo(data, eps, min_pts)

    # 4. Print Summary
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)
    print(f"\n--- Final Summary ---")
    print(f"Estimated number of clusters: {n_clusters}")
    print(f"Estimated number of noise points: {n_noise}")
    print(f"Total core points identified: {len(core_point_indices)}")
    print(f"Check the '{output_dir}' directory for step-by-step images.")